# Sensitivity Analysis for Greenwashing Risk Assessment

## Overview
This module extends the main greenwashing scoring with comprehensive sensitivity analysis to test methodological robustness. It examines how sub-component weight variations affect final scores and company rankings, addressing theoretical uncertainty about optimal weight allocations within communication dimensions.

## Sensitivity Scenarios (17 total)
- **Baseline**: Literature-based weights as established in main analysis
- **Systematic variations**: ±10% adjustments for most components, ±5% for substantiation weakness (preserving Evidence ≥ Aspirational > Quantified hierarchy)
- **Component coverage**: All sub-components within sentiment, green terms, substantiation, language vagueness, temporal orientation, and reporting consistency dimensions
- **Hierarchy preservation**: Weight modifications maintain theoretical priorities while testing robustness boundaries

## Enhanced Ensemble Processing
**Per-scenario analysis**: Each of 17 scenarios runs through complete ensemble methodology with all 59,881 valid weight combinations
**Computational scope**: Total analysis spans ~1 million weight combination tests across all scenarios
**Consistency maintenance**: Same methodology as main analysis ensures comparable results

## Sensitivity-Specific Metrics Added

### Ranking Stability Analysis
- **R̄S (R-bar-S)**: Average ranking shift across scenarios compared to baseline rankings
- **Robustness classification**: R̄S < 2.0 (high), 2.0-4.0 (moderate), >4.0 (low robustness)
- **Cross-year tracking**: Ranking sensitivity measured separately for 2021 and 2022

### Score Variability Metrics  
- **MAD (Mean Absolute Deviation)**: Average score shifts from baseline values across scenarios
- **CV (Coefficient of Variation)**: Standardized variability measure enabling comparison across score levels
- **Score Range**: Maximum uncertainty span (max score - min score) for each company
- **High sensitivity threshold**: Companies with CV >15% or score range >10 points flagged for careful interpretation

### Company-Level Sensitivity Classification
- **Sensitivity_Level**: High/Moderate/Low classification based on combined CV, score range, and ranking shift metrics
- **Max_Rank_Shift**: Largest ranking change experienced across all scenarios
- **Avg_Rank_Shift**: Average ranking movement indicating overall positional stability

## Statistical Framework
Follows Saisana et al. (2005) methodology for composite index sensitivity testing, distinguishing between components affecting rankings versus absolute score values. Results identify both methodological elements causing largest disruptions and companies requiring cautious interpretation due to weight sensitivity.

## Key Sensitivity Output Variables
- **Sens_CV_Percent**: Coefficient of variation across weight scenarios (%)
- **Sens_Score_Range**: Score range across all weight scenarios  
- **Sens_MAD**: Mean absolute deviation from baseline
- **Sens_Avg_Rank_Shift**: Average ranking shift across scenarios
- **Sens_Max_Rank_Shift**: Maximum ranking shift across scenarios
- **Sens_Sensitivity_Level**: Sensitivity classification (High/Moderate/Low)

This analysis validates GRAT robustness by revealing which sub-component weights most influence results and identifying companies whose assessments remain stable versus those requiring methodological attention.

## Loading and preparing data

In [ ]:
import pandas as pd
import os

# Load all Excel files
df_density = pd.read_excel('data/NLP/Results/Communication_Score_df_Density.xlsx')
df_context = pd.read_excel('data/NLP/Results/Communication_Score_df_Context.xlsx')
df_similarity = pd.read_excel('data/NLP/Results/Similarity/similarity_analysis_results.xlsx')
df_sentiment = pd.read_excel('data/NLP/Results/Overall_Sentiment_Analysis.xlsx')
df_hedge_vague = pd.read_excel('data/NLP/Results/Communication_Score_df_Hedge_Vague.xlsx')
df_topics = pd.read_excel('data/NLP/Results/Communication_Score_df_Topics.xlsx')
df_topic_sentiment = pd.read_excel('data/NLP/Results/Topic_Weighted_Sentiment_Analysis.xlsx')

# For each df print the name of the first column
dataframes = [df_density, df_context, df_similarity, df_sentiment, df_hedge_vague, df_topics, df_topic_sentiment]
for df in dataframes:
    first_col = df.columns[0]
    print(f"First column: {first_col}")

In [ ]:
# Extract relevant metrics from each dataframe
density_metrics = df_density[['organization', 'year', 'gt_freq_pct', 'unique_gt_relative']].copy()

context_metrics = df_context[['organization', 'year', 'temporal_past_pct', 'temporal_present_pct', 
                             'temporal_future_pct', 'quantification_intensity_score', 
                             'evidence_intensity_score', 'aspirational_intensity_score']].copy()

similarity_metrics = df_similarity[['Company', 'TFIDF_Doc', 'Jaccard', 'SpaCy_HighSim_Ratio', 'SpaCy_Avg_Similarity']].copy()
similarity_metrics.rename(columns={'Company': 'organization'}, inplace=True)

sentiment_metrics = df_sentiment[['organization', 'year', 'avg_sentiment_score', 'sentiment_confidence', 'opportunity_ratio', 'risk_ratio'
]].copy()

hedge_vague_metrics = df_hedge_vague[['organization', 'year', 'hedge_intensity_score', 
                                     'vague_intensity_score', 'commitment_timeline_pct', 'total_unclear_density',
                                     'combined_intensity_score']].copy()

topics_metrics = df_topics[['organization', 'year', 'renewable_energy_density', 'climate_emissions_density']].copy()

topic_sentiment_metrics = df_topic_sentiment[['organization', 'year', 'renewable_energy_avg_sentiment', 
                                             'climate_emissions_avg_sentiment']].copy()

# Standardize organization names by replacing underscores with spaces
density_metrics['organization'] = density_metrics['organization'].str.replace('_', ' ')
context_metrics['organization'] = context_metrics['organization'].str.replace('_', ' ')
sentiment_metrics['organization'] = sentiment_metrics['organization'].str.replace('_', ' ')
hedge_vague_metrics['organization'] = hedge_vague_metrics['organization'].str.replace('_', ' ')
topics_metrics['organization'] = topics_metrics['organization'].str.replace('_', ' ')
topic_sentiment_metrics['organization'] = topic_sentiment_metrics['organization'].str.replace('_', ' ')

In [ ]:
# Merge all dataframes
df = density_metrics.merge(context_metrics, on=['organization', 'year'], how='outer')
df = df.merge(sentiment_metrics, on=['organization', 'year'], how='outer')
df = df.merge(hedge_vague_metrics, on=['organization', 'year'], how='outer')
df = df.merge(topics_metrics, on=['organization', 'year'], how='outer')
df = df.merge(topic_sentiment_metrics, on=['organization', 'year'], how='outer')
df = df.merge(similarity_metrics, on='organization', how='outer')

# Rename organization column to Organization
df.rename(columns={'organization': 'Organization'}, inplace=True)

In [ ]:
# Function to extract first word or apply exceptions
def simplify_org_name(name):
    if name == 'Polska Grupa Energetyczna PGE SA':
        return 'PGE'
    elif name == 'AKENERJİ ELEKTRİK ÜRETİM A.Ş.':
        return 'Akenerji'
    else:
        return name.split()[0]

# Apply the function to the 'Organization' column
df.loc[:, 'Organization'] = df['Organization'].apply(simplify_org_name)

In [ ]:
# Check for NaN values and show which metrics and organizations have them
print("MISSING DATA ANALYSIS")
print("=" * 50)

# Get columns with NaN values
cols_with_nan = df.columns[df.isnull().any()].tolist()

if cols_with_nan:
    print(f"Metrics with NaN values: {cols_with_nan}")
    print()
    
    for col in cols_with_nan:
        nan_rows = df[df[col].isnull()]
        if not nan_rows.empty:
            print(f"Metric: {col}")
            print(f"Organizations with missing data:")
            for _, row in nan_rows.iterrows():
                if 'year' in df.columns:
                    print(f"  - {row['Organization']} ({row['year']})")
                else:
                    print(f"  - {row['Organization']}")
            print()
else:
    print("No NaN values found in the dataset.")

# Summary statistics
total_cells = df.shape[0] * df.shape[1]
nan_cells = df.isnull().sum().sum()
print(f"Total cells: {total_cells}")
print(f"NaN cells: {nan_cells}")
print(f"Missing data percentage: {(nan_cells/total_cells)*100:.2f}%")

In [ ]:
# Load cleaned Excel with new structure
ensemble_perf = pd.read_excel('data/Performance/ensemble_performance_scores.xlsx')

# Confirm actual column names
print("Columns:", ensemble_perf.columns)

# Fix name consistency (Ørsted → Orsted)
ensemble_perf['Organization'] = ensemble_perf['Organization'].replace('Ørsted', 'Orsted')

# Rename median_score to Performance_Score (data is already in long format)
ensemble_perf = ensemble_perf.rename(columns={'median_score': 'Performance_Score'})

# Merge with your main DataFrame
df = df.merge(ensemble_perf[['Organization', 'year', 'Performance_Score']], on=['Organization', 'year'], how='left')

# Diagnostics
print("Performance scores added to df")
print(f"Shape: {df.shape}")
print(f"Performance scores available: {df['Performance_Score'].notna().sum()}/{len(df)}")

## Helper function

In [ ]:
# Normalize scores per year (0-100 scale)
def normalize_by_year(df, column):
    normalized_col = f"{column}"
    df[normalized_col] = df.groupby('year')[column].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) * 100 if x.max() != x.min() else 50
    )
    return normalized_col

## Aditional metrics

In [ ]:
# Normalize similarity values
normalize_by_year(df, 'TFIDF_Doc')
normalize_by_year(df, 'Jaccard')
normalize_by_year(df, 'SpaCy_Avg_Similarity')

# Create similarity combined score (average of three similarity metrics)
df['similarity_combined'] = (
    (df['TFIDF_Doc'] + 
     df['Jaccard'] + 
     df['SpaCy_Avg_Similarity']) / 3
).round(2)


# Calculate an other additional metric
# Add future vs past+present ratio
df['future_vs_past_present_ratio'] = (
    df['temporal_future_pct'] / 
    (df['temporal_past_pct'] + df['temporal_present_pct'])
).round(2)

## Create df used for greenwashing score calculation

In [ ]:
# Create a copy of df
greenwashing_df = df.copy()

# Drop specified columns
columns_to_drop = [
    'temporal_past_pct', 'temporal_present_pct', 'temporal_future_pct',
    'TFIDF_Doc', 'Jaccard', 'SpaCy_Avg_Similarity',
    'sentiment_confidence', 'opportunity_ratio', 'risk_ratio',
    'total_unclear_density', 'renewable_energy_density', 'climate_emissions_density'
]
greenwashing_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Normalize all numeric columns except 'year' and 'Performance_Score'
numeric_cols = greenwashing_df.select_dtypes(include='number').columns
numeric_cols = [col for col in numeric_cols if col != 'year'] # was: ...if col not in ['year', 'Performance_Score'] OR: if col != 'year'

for col in numeric_cols:
    normalize_by_year(greenwashing_df, col)


# Calculation of Greenwashing Components

In [ ]:
# Combined sentiment score

# Calculate combined sentiment score
greenwashing_df['combined_sentiment_score'] = (
    0.6 * greenwashing_df['avg_sentiment_score'] +     
    0.2 * greenwashing_df['renewable_energy_avg_sentiment'] +
    0.2 * greenwashing_df['climate_emissions_avg_sentiment']
).round(2)


In [ ]:
# Combined green term score

# Calculate combined sentiment score
greenwashing_df['combined_green_score'] = (
    0.7 * greenwashing_df['gt_freq_pct'] +     
    0.3 * greenwashing_df['unique_gt_relative']
).round(2)

In [ ]:
# Calculate communication score
greenwashing_df['Green_Com_Score'] = (
    0.4 * greenwashing_df['combined_green_score'] +
    0.6 * greenwashing_df['combined_sentiment_score']
).round(2)

# Normalize Green_Com_Score per year to 0–100 scale
normalize_by_year(greenwashing_df, 'Green_Com_Score')

print("Communication score created")
print(f"Mean score: {greenwashing_df['Green_Com_Score'].mean():.2f}")
print(f"Score range: {greenwashing_df['Green_Com_Score'].min():.2f} - {greenwashing_df['Green_Com_Score'].max():.2f}")

In [ ]:
# Calculate absolute gap between performance and communication
greenwashing_df['Greenwashing_Risk_Abs'] = (
    greenwashing_df['Green_Com_Score'] - greenwashing_df['Performance_Score']
).round(2)

# Calculate yearly medians for classic greenwashing pattern
yearly_medians = greenwashing_df.groupby('year').agg({
    'Performance_Score': 'median',
    'Green_Com_Score': 'median'
})

# Identify classic greenwashing pattern
greenwashing_df['Classic_Greenwashing'] = greenwashing_df.apply(
    lambda row: (row['Green_Com_Score'] > yearly_medians.loc[row['year'], 'Green_Com_Score']) and 
                (row['Performance_Score'] < yearly_medians.loc[row['year'], 'Performance_Score']), 
    axis=1
)

# Apply 1.5x amplifier for classic greenwashing
greenwashing_df['Amplified_Score'] = greenwashing_df.apply(
    lambda row: row['Greenwashing_Risk_Abs'] * 1.5 if row['Classic_Greenwashing'] else row['Greenwashing_Risk_Abs'],
    axis=1
).round(2)

# Normalize to 0–100 scale scale within each year
normalize_by_year(greenwashing_df, 'Amplified_Score')
greenwashing_df['Greenwashing_Score'] = greenwashing_df['Amplified_Score'].round(2)

# Clean up intermediate columns
greenwashing_df = greenwashing_df.drop(['Classic_Greenwashing'], axis=1) # kept 'Base_Hybrid_Score', 'Amplified_Score'

print("Greenwashing score calculated")
print(f"Mean score: {greenwashing_df['Greenwashing_Score'].mean():.2f}")
print(f"Companies with classic greenwashing pattern: {greenwashing_df.apply(lambda row: (row['Green_Com_Score'] > yearly_medians.loc[row['year'], 'Green_Com_Score']) and (row['Performance_Score'] < yearly_medians.loc[row['year'], 'Performance_Score']), axis=1).sum()}")

# 2021 highest scores
print(f"\n2021 HIGHEST GREENWASHING SCORES:")
highest_2021 = greenwashing_df[greenwashing_df['year'] == 2021].nlargest(10, 'Greenwashing_Score')[['Organization', 'Greenwashing_Score']]
print(highest_2021.to_string(index=False))

# 2022 highest scores  
print(f"\n2022 HIGHEST GREENWASHING SCORES:")
highest_2022 = greenwashing_df[greenwashing_df['year'] == 2022].nlargest(10, 'Greenwashing_Score')[['Organization', 'Greenwashing_Score']]
print(highest_2022.to_string(index=False))

# Average scores per company across both years
print(f"\nHIGHEST AVERAGE GREENWASHING SCORES (2021-2022):")
company_averages = greenwashing_df.groupby('Organization')['Greenwashing_Score'].agg(['mean', 'count']).reset_index()
company_averages.columns = ['Organization', 'Avg_Greenwashing_Score', 'Years_Count']
company_averages = company_averages[company_averages['Years_Count'] == 2]  # Only companies with data for both years
highest_avg = company_averages.nlargest(10, 'Avg_Greenwashing_Score')[['Organization', 'Avg_Greenwashing_Score']]
highest_avg['Avg_Greenwashing_Score'] = highest_avg['Avg_Greenwashing_Score'].round(2)
print(highest_avg.to_string(index=False))

print(f"\nCompanies with data for both years: {len(company_averages)}")
print(f"Average greenwashing score across all companies (both years): {company_averages['Avg_Greenwashing_Score'].mean():.2f}")

In [ ]:
# Save performance communication gap data to Excel
import os

# Create directory if it doesn't exist
os.makedirs('data/Greenwashing Results', exist_ok=True)

# Save the dataframe to Excel
greenwashing_df.to_excel('data/Greenwashing Results/performance_communication_gap.xlsx', 
                        index=False)

print("Performance communication gap data saved to: data/Greenwashing Results/performance_communication_gap.xlsx")
print(f"Saved {len(greenwashing_df)} rows and {len(greenwashing_df.columns)} columns")

In [ ]:
# Greenwashing Score Quadrant Visualization
# Communication Score vs Performance Score Analysis

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Calculate medians for thresholds
comm_median = greenwashing_df['Green_Com_Score'].median()
perf_median = greenwashing_df['Performance_Score'].median()

# Create quadrant classification
def classify_quadrant(row):
    comm_high = row['Green_Com_Score'] >= comm_median
    perf_high = row['Performance_Score'] >= perf_median
    
    if comm_high and not perf_high:
        return 'Potential_Greenwashing'
    elif comm_high and perf_high:
        return 'Green_Leaders'
    elif not comm_high and not perf_high:
        return 'Laggards'
    else:
        return 'Under_Communicators'

greenwashing_df['Quadrant'] = greenwashing_df.apply(classify_quadrant, axis=1)

# Define colors for quadrants
colors = {'Potential_Greenwashing': 'red', 'Green_Leaders': 'green', 
          'Laggards': 'gray', 'Under_Communicators': 'blue'}

# Filter data by year
df_2021 = greenwashing_df[greenwashing_df['year'] == 2021]
df_2022 = greenwashing_df[greenwashing_df['year'] == 2022]

# Create the visualization function
def create_quadrant_plot(data, year, ax):
    """Create a quadrant plot for the given year"""
    
    # Calculate year-specific medians
    year_comm_median = data['Green_Com_Score'].median()
    year_perf_median = data['Performance_Score'].median()
    
    # Plot each quadrant
    for quadrant in data['Quadrant'].unique():
        if pd.isna(quadrant):
            continue
        subset = data[data['Quadrant'] == quadrant]
        ax.scatter(subset['Performance_Score'], subset['Green_Com_Score'], 
                  c=colors[quadrant], label=quadrant.replace('_', ' '), 
                  alpha=0.7, s=100, edgecolors='black', linewidth=0.5)
    
    # Add company name annotations
    for _, row in data.iterrows():
        if pd.notna(row['Performance_Score']) and pd.notna(row['Green_Com_Score']):
            ax.annotate(row['Organization'], 
                       (row['Performance_Score'], row['Green_Com_Score']),
                       xytext=(5, 5), textcoords='offset points', 
                       fontsize=10, alpha=0.9, fontweight='bold')
    
    # Add median lines
    ax.axvline(year_perf_median, color='gray', linestyle='--', alpha=0.5, linewidth=2)
    ax.axhline(year_comm_median, color='gray', linestyle='--', alpha=0.5, linewidth=2)
    
    # Add quadrant labels
    perf_range = ax.get_xlim()
    comm_range = ax.get_ylim()
    
    # Potential Greenwashing (Top Left)
    ax.text(perf_range[0] + (year_perf_median - perf_range[0])/2, 
            year_comm_median + (comm_range[1] - year_comm_median)/2,
            'Potential\nGreenwashing', ha='center', va='center', fontsize=11, 
            bbox=dict(boxstyle="round,pad=0.3", facecolor='lightcoral', alpha=0.7))
    
    # Green Leaders (Top Right)
    ax.text(year_perf_median + (perf_range[1] - year_perf_median)/2, 
            year_comm_median + (comm_range[1] - year_comm_median)/2,
            'Green\nLeaders', ha='center', va='center', fontsize=11, 
            bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgreen', alpha=0.7))
    
    # Laggards (Bottom Left)
    ax.text(perf_range[0] + (year_perf_median - perf_range[0])/2, 
            comm_range[0] + (year_comm_median - comm_range[0])/2,
            'Laggards', ha='center', va='center', fontsize=11, 
            bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgray', alpha=0.7))
    
    # Under Communicators (Bottom Right)
    ax.text(year_perf_median + (perf_range[1] - year_perf_median)/2, 
            comm_range[0] + (year_comm_median - comm_range[0])/2,
            'Under\nCommunicators', ha='center', va='center', fontsize=11, 
            bbox=dict(boxstyle="round,pad=0.3", facecolor='lightblue', alpha=0.7))
    
    # Customize axes
    ax.set_xlabel('Performance Score', fontsize=14)
    ax.set_ylabel('Communication Score', fontsize=14)
    ax.set_title(f'Greenwashing Detection {year}', fontsize=16, fontweight='bold')
    ax.legend(loc='upper left', bbox_to_anchor=(0, 1))
    ax.grid(True, alpha=0.3)

# Create figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(24, 10))

# FIRST GRAPH: 2021
if len(df_2021) > 0:
    create_quadrant_plot(df_2021, 2021, axes[0])
else:
    axes[0].text(0.5, 0.5, 'No data available for 2021', 
                ha='center', va='center', transform=axes[0].transAxes, fontsize=14)
    axes[0].set_title('Greenwashing Detection 2021', fontsize=16, fontweight='bold')

# SECOND GRAPH: 2022
if len(df_2022) > 0:
    create_quadrant_plot(df_2022, 2022, axes[1])
else:
    axes[1].text(0.5, 0.5, 'No data available for 2022', 
                ha='center', va='center', transform=axes[1].transAxes, fontsize=14)
    axes[1].set_title('Greenwashing Detection 2022', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

# Print detailed quadrant analysis
print("GREENWASHING QUADRANT ANALYSIS")
print("=" * 60)

for year in [2021, 2022]:
    year_data = greenwashing_df[greenwashing_df['year'] == year]
    if len(year_data) > 0:
        print(f"\n{year} QUADRANT DISTRIBUTION:")
        print("-" * 40)
        
        quadrant_counts = year_data['Quadrant'].value_counts()
        for quadrant, count in quadrant_counts.items():
            if pd.notna(quadrant):
                print(f"{quadrant.replace('_', ' ')}: {count} companies")
        
        print(f"\nMedian Communication Score: {year_data['Green_Com_Score'].median():.2f}")
        print(f"Median Performance Score: {year_data['Performance_Score'].median():.2f}")
        
        # Show companies in each quadrant with their scores
        print(f"\nDETAILED BREAKDOWN ({year}):")
        for quadrant in ['Potential_Greenwashing', 'Green_Leaders', 'Laggards', 'Under_Communicators']:
            quad_data = year_data[year_data['Quadrant'] == quadrant]
            if len(quad_data) > 0:
                print(f"\n{quadrant.replace('_', ' ')} ({len(quad_data)} companies):")
                for _, row in quad_data.iterrows():
                    print(f"  {row['Organization']}: Com={row['Green_Com_Score']:.1f}, Perf={row['Performance_Score']:.1f}, Risk={row['Greenwashing_Score']:.1f}")

# Analysis of companies that changed quadrants
print(f"\n" + "=" * 60)
print("QUADRANT MOVEMENT ANALYSIS (2021 → 2022)")
print("=" * 60)

companies_both_years = set(df_2021['Organization'].unique()) & set(df_2022['Organization'].unique())

movements = []
for company in companies_both_years:
    quad_2021 = df_2021[df_2021['Organization'] == company]['Quadrant'].iloc[0]
    quad_2022 = df_2022[df_2022['Organization'] == company]['Quadrant'].iloc[0]
    
    if quad_2021 != quad_2022:
        movements.append({
            'Company': company,
            'From': quad_2021,
            'To': quad_2022
        })

if movements:
    print(f"\nCompanies that changed quadrants: {len(movements)}")
    for movement in movements:
        print(f"{movement['Company']}: {movement['From'].replace('_', ' ')} → {movement['To'].replace('_', ' ')}")
else:
    print("\nNo companies changed quadrants between 2021 and 2022")

# Summary statistics
print(f"\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

print(f"Overall median Communication Score: {greenwashing_df['Green_Com_Score'].median():.2f}")
print(f"Overall median Performance Score: {greenwashing_df['Performance_Score'].median():.2f}")
print(f"Overall median Greenwashing Score: {greenwashing_df['Greenwashing_Score'].median():.2f}")

print(f"\nCorrelation between Communication and Performance: {greenwashing_df['Green_Com_Score'].corr(greenwashing_df['Performance_Score']):.3f}")
print(f"Correlation between Communication and Greenwashing Risk: {greenwashing_df['Green_Com_Score'].corr(greenwashing_df['Greenwashing_Score']):.3f}")
print(f"Correlation between Performance and Greenwashing Risk: {greenwashing_df['Performance_Score'].corr(greenwashing_df['Greenwashing_Score']):.3f}")

# Print companies with highest risk scores in each quadrant
print(f"\n" + "=" * 60)
print("HIGHEST RISK COMPANIES BY QUADRANT")
print("=" * 60)

for quadrant in ['Potential_Greenwashing', 'Green_Leaders', 'Laggards', 'Under_Communicators']:
    quad_data = greenwashing_df[greenwashing_df['Quadrant'] == quadrant]
    if len(quad_data) > 0:
        top_risk = quad_data.nlargest(3, 'Greenwashing_Score')
        print(f"\n{quadrant.replace('_', ' ')} - Top Risk:")
        for _, row in top_risk.iterrows():
            print(f"  {row['Organization']} ({row['year']}): Risk={row['Greenwashing_Score']:.1f}")

### Additional components for greenwashing

In [ ]:
# ==========================================
# COMPONENT 1: SUBSTANTIATION WEAKNESS SCORE
# Quantification (30%) + Evidence (35%) + Aspirational (35%)
# Higher scores = higher greenwashing risk
# ==========================================

greenwashing_df['Substantiation_Weakness'] = (
    0.30 * (100 - greenwashing_df['quantification_intensity_score']) +     # Reversed: higher quantification = lower risk
    0.35 * (100 - greenwashing_df['evidence_intensity_score']) +          # Reversed: higher evidence = lower risk  
    0.35 * greenwashing_df['aspirational_intensity_score']                # Normal: higher aspirational = higher risk
).round(2)



In [ ]:
# COMPONENT 2: LANGUAGE VAGUENESS SCORE
# Vague (70%) + Hedge (30%)
# Higher scores = higher greenwashing risk
# ==========================================

greenwashing_df['Language_Vagueness'] = (
    0.70 * greenwashing_df['vague_intensity_score'] +                     # Normal: higher vagueness = higher risk
    0.30 * (100 - greenwashing_df['hedge_intensity_score'])               # Reversed: higher hedging = lower risk (per UCLA study)
).round(2)



In [ ]:
# COMPONENT 3: TEMPORAL ORIENTATION SCORE  
# Future orientation (60%) + Timeline specificity (40%)
# Higher scores = higher greenwashing risk
# ==========================================

greenwashing_df['Temporal_Orientation'] = (
    0.60 * greenwashing_df['future_vs_past_present_ratio'] +             # Normal: higher future focus = higher risk
    0.40 * (100 - greenwashing_df['commitment_timeline_pct'])            # Reversed: more specific timelines = lower risk
).round(2)



In [ ]:
# COMPONENT 4: REPORTING CONSISTENCY SCORE
# Overall similarity (70%) + High similarity ratio (30%)  
# Higher scores = higher greenwashing risk
# ==========================================

greenwashing_df['Reporting_Consistency'] = (
    0.70 * greenwashing_df['similarity_combined'] +                      # Normal: higher similarity = higher risk
    0.30 * greenwashing_df['SpaCy_HighSim_Ratio']                       # Normal: more identical sentences = higher risk
).round(2)



In [ ]:
# STANDARDIZE EACH COMPONENT BY YEAR
# ==========================================

normalize_by_year(greenwashing_df, 'Substantiation_Weakness')
normalize_by_year(greenwashing_df, 'Language_Vagueness')
normalize_by_year(greenwashing_df, 'Temporal_Orientation')
normalize_by_year(greenwashing_df, 'Reporting_Consistency')


# ==========================================
# SUMMARY STATISTICS BY YEAR
# ==========================================

print("GREENWASHING COMPONENTS SUMMARY BY YEAR")
print("="*60)

# Component 1: Substantiation Weakness
print(f"\nSUBSTANTIATION WEAKNESS")
for year in [2021, 2022]:
    year_data = greenwashing_df[greenwashing_df['year'] == year]
    print(f"\n{year}:")
    print(f"  Mean Score: {year_data['Substantiation_Weakness'].mean():.2f}")
    print(f"  Highest Risk Companies:")
    top_subst = year_data.nlargest(5, 'Substantiation_Weakness')[['Organization', 'Substantiation_Weakness']]
    for i, (idx, row) in enumerate(top_subst.iterrows(), 1):
        print(f"    {i}. {row['Organization']}: {row['Substantiation_Weakness']:.2f}")

# Component 2: Language Vagueness
print(f"\nLANGUAGE VAGUENESS")
for year in [2021, 2022]:
    year_data = greenwashing_df[greenwashing_df['year'] == year]
    print(f"\n{year}:")
    print(f"  Mean Score: {year_data['Language_Vagueness'].mean():.2f}")
    print(f"  Highest Risk Companies:")
    top_lang = year_data.nlargest(5, 'Language_Vagueness')[['Organization', 'Language_Vagueness']]
    for i, (idx, row) in enumerate(top_lang.iterrows(), 1):
        print(f"    {i}. {row['Organization']}: {row['Language_Vagueness']:.2f}")

# Component 3: Temporal Orientation  
print(f"\nTEMPORAL ORIENTATION")
for year in [2021, 2022]:
    year_data = greenwashing_df[greenwashing_df['year'] == year]
    print(f"\n{year}:")
    print(f"  Mean Score: {year_data['Temporal_Orientation'].mean():.2f}")
    print(f"  Highest Risk Companies:")
    top_temp = year_data.nlargest(5, 'Temporal_Orientation')[['Organization', 'Temporal_Orientation']]
    for i, (idx, row) in enumerate(top_temp.iterrows(), 1):
        print(f"    {i}. {row['Organization']}: {row['Temporal_Orientation']:.2f}")

# Component 4: Reporting Consistency
print(f"\nREPORTING CONSISTENCY")
for year in [2021, 2022]:
    year_data = greenwashing_df[greenwashing_df['year'] == year]
    print(f"\n{year}:")
    print(f"  Mean Score: {year_data['Reporting_Consistency'].mean():.2f}")
    print(f"  Highest Risk Companies:")
    top_cons = year_data.nlargest(5, 'Reporting_Consistency')[['Organization', 'Reporting_Consistency']]
    for i, (idx, row) in enumerate(top_cons.iterrows(), 1):
        print(f"    {i}. {row['Organization']}: {row['Reporting_Consistency']:.2f}")

print(f"\n" + "="*60)
print("Component calculations and standardization complete!")
print("Higher scores indicate higher greenwashing risk")

In [ ]:
greenwashing_df

### Export and save

In [ ]:
# Cell: Create Comprehensive Greenwashing Results Export

print("CREATING COMPREHENSIVE GREENWASHING RESULTS")
print("=" * 60)

# ==========================================
# STEP 1: Create comprehensive results DataFrame with all variables
# ==========================================

print("Preparing comprehensive greenwashing dataset...")

# Create comprehensive dataset with all variables and clear naming
comprehensive_greenwashing = pd.DataFrame({
    # ============= IDENTIFICATION =============
    'Company': greenwashing_df['Organization'],
    'Year': greenwashing_df['year'],
    
    # ============= MAIN SCORES =============
    'Performance_Communication_Gap_Score': greenwashing_df['Greenwashing_Score'],
    'Performance_Score': greenwashing_df['Performance_Score'], 
    'Green_Communication_Score': greenwashing_df['Green_Com_Score'],
    'Performance_Communication_Absolute_Gap': greenwashing_df['Greenwashing_Risk_Abs'],
    # 'Performance_Communication_Gap_Amplified': greenwashing_df['Amplified_Score'],
    
    # ============= COMPONENT SCORES (0-100 each) =============
    'Component_Substantiation_Weakness_Score': greenwashing_df['Substantiation_Weakness'],
    'Component_Language_Vagueness_Score': greenwashing_df['Language_Vagueness'], 
    'Component_Temporal_Orientation_Score': greenwashing_df['Temporal_Orientation'],
    'Component_Reporting_Consistency_Score': greenwashing_df['Reporting_Consistency'],
    
    # ============= GREEN TERM ANALYSIS =============
    'Green_Terms_Frequency_Pct': df['gt_freq_pct'],
    'Green_Terms_Unique_Relative': df['unique_gt_relative'],
    'Green_Terms_Combined_Score': greenwashing_df['combined_green_score'],
    
    # ============= SENTIMENT ANALYSIS =============
    'Overall_Sentiment_Score': df['avg_sentiment_score'],
    'Renewable_Energy_Sentiment': df['renewable_energy_avg_sentiment'],
    'Climate_Emissions_Sentiment': df['climate_emissions_avg_sentiment'],
    'Combined_Sentiment_Score': greenwashing_df['combined_sentiment_score'],
    
    # ============= CONTEXT & SUBSTANTIATION =============
    # Quantification, Evidence, Aspirational (used in Substantiation Weakness)
    'Quantification_Intensity_Score': df['quantification_intensity_score'],
    'Evidence_Intensity_Score': df['evidence_intensity_score'], 
    'Aspirational_Intensity_Score': df['aspirational_intensity_score'],
    
    # ============= LANGUAGE VAGUENESS METRICS =============
    # Hedging, Vague language (used in Language Vagueness)
    'Hedge_Intensity_Score': df['hedge_intensity_score'],
    'Vague_Intensity_Score': df['vague_intensity_score'],
    'Combined_Unclear_Intensity_Score': greenwashing_df['combined_intensity_score'],
    'Commitment_Timeline_Pct': df['commitment_timeline_pct'],
    
    # ============= TEMPORAL ORIENTATION METRICS =============
    # Future vs past/present focus (used in Temporal Orientation)
    'Future_vs_Past_Present_Ratio': df['future_vs_past_present_ratio'],
    
    # ============= SIMILARITY/CONSISTENCY METRICS =============
    # Document similarity (used in Reporting Consistency)
    'TFIDF_Document_Similarity': df['TFIDF_Doc'],
    'Jaccard_Similarity': df['Jaccard'],
    'SpaCy_Average_Similarity': df['SpaCy_Avg_Similarity'],
    'SpaCy_High_Similarity_Ratio': df['SpaCy_HighSim_Ratio'],
    'Similarity_Combined_Score': greenwashing_df['similarity_combined'],
})

# Round all numeric columns
numeric_columns = comprehensive_greenwashing.select_dtypes(include=[np.number]).columns
comprehensive_greenwashing[numeric_columns] = comprehensive_greenwashing[numeric_columns].round(3)

# Sort by Company and Year
comprehensive_greenwashing = comprehensive_greenwashing.sort_values(['Company', 'Year']).reset_index(drop=True)

print(f"✓ Comprehensive dataset created: {comprehensive_greenwashing.shape}")

# ==========================================
# STEP 2: Create component breakdown analysis
# ==========================================

print("Creating component breakdown analysis...")

# Component weights used in calculations (for documentation)
component_weights = pd.DataFrame({
    'Component': [
        'Substantiation Weakness - Quantification',
        'Substantiation Weakness - Evidence', 
        'Substantiation Weakness - Aspirational',
        'Language Vagueness - Vague Language',
        'Language Vagueness - Hedge Language',
        'Temporal Orientation - Future vs Past/Present',
        'Temporal Orientation - Timeline Specificity',
        'Reporting Consistency - Overall Similarity',
        'Reporting Consistency - High Similarity Ratio',
        'Green Communication - Green Terms',
        'Green Communication - Sentiment'
    ],
    'Weight': [0.30, 0.35, 0.35, 0.70, 0.30, 0.60, 0.40, 0.70, 0.30, 0.40, 0.60],
    'Direction': [
        'Reversed (higher quantification = lower risk)',
        'Reversed (higher evidence = lower risk)',
        'Normal (higher aspirational = higher risk)',
        'Normal (higher vagueness = higher risk)',
        'Reversed (higher hedging = lower risk)',
        'Normal (higher future focus = higher risk)',
        'Reversed (more specific timelines = lower risk)',
        'Normal (higher similarity = higher risk)',
        'Normal (more identical sentences = higher risk)',
        'Normal (higher green terms = higher communication)',
        'Normal (higher sentiment = higher communication)'
    ]
})

# ==========================================
# STEP 3: Create summary statistics by year
# ==========================================

print("Calculating summary statistics by year...")

summary_stats_2021 = comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021].describe()
summary_stats_2022 = comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022].describe()

# Component score means by year
component_summary = pd.DataFrame({
    'Component': [
        'Greenwashing Risk Score',
        'Green Communication Score', 
        'Substantiation Weakness',
        'Language Vagueness',
        'Temporal Orientation',
        'Reporting Consistency'
    ],
    'Mean_2021': [
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Performance_Communication_Gap_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Green_Communication_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Component_Substantiation_Weakness_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Component_Language_Vagueness_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Component_Temporal_Orientation_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Component_Reporting_Consistency_Score'].mean()
    ],
    'Mean_2022': [
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Performance_Communication_Gap_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Green_Communication_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Component_Substantiation_Weakness_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Component_Language_Vagueness_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Component_Temporal_Orientation_Score'].mean(),
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Component_Reporting_Consistency_Score'].mean()
    ]
}).round(2)

# ==========================================
# STEP 4: Export to Excel with multiple tabs
# ==========================================

print("\nExporting comprehensive greenwashing results to Excel...")
output_path = "data/Greenwashing Results/comprehensive_greenwashing_results.xlsx"

try:
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Main comprehensive dataset
        comprehensive_greenwashing.to_excel(writer, sheet_name='Comprehensive_Results', index=False)
        
        # Separate years for easier analysis
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021].to_excel(
            writer, sheet_name='Results_2021', index=False)
        comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022].to_excel(
            writer, sheet_name='Results_2022', index=False)
        
        # Component weights and methodology
        component_weights.to_excel(writer, sheet_name='Component_Methodology', index=False)
        
        # Component summary statistics
        component_summary.to_excel(writer, sheet_name='Component_Summary', index=False)
        
        # Detailed statistics by year
        summary_stats_2021.to_excel(writer, sheet_name='Stats_2021')
        summary_stats_2022.to_excel(writer, sheet_name='Stats_2022')
        
        # Data dictionary
        data_dict = pd.DataFrame({
            'Column_Name': comprehensive_greenwashing.columns,
            'Category': [
                'Identification' if col in ['Company', 'Year'] else
                'Main Scores' if any(x in col for x in ['Performance_Communication_Gap_Score', 'Performance_Score', 'Green_Communication_Score', 'Absolute_Gap', 'Amplified']) else
                'Component Scores' if 'Component_' in col else
                'Green Terms Analysis' if any(x in col for x in ['Green_Terms', 'combined_green_score']) else
                'Sentiment Analysis' if any(x in col for x in ['Sentiment', 'sentiment']) else
                'Substantiation Metrics' if any(x in col for x in ['Quantification', 'Evidence', 'Aspirational']) else
                'Language Vagueness Metrics' if any(x in col for x in ['Hedge', 'Vague', 'Unclear', 'Timeline']) else
                'Temporal Metrics' if 'Future_vs_Past' in col else
                'Similarity Metrics' if any(x in col for x in ['Similarity', 'TFIDF', 'Jaccard', 'SpaCy']) else
                'Other' for col in comprehensive_greenwashing.columns
            ],
            'Description': [
                'Company identifier' if col == 'Company' else
                'Year (2021 or 2022)' if col == 'Year' else
                '(Amplified) Performance Communication Ga[ score (0-100, higher = more risk)' if col == 'Performance_Communication_Gap_Score' else
                'Environmental performance score from ensemble analysis' if col == 'Performance_Score' else
                'Green communication intensity score (0-100)' if col == 'Green_Communication_Score' else
                'Absolute gap between communication and performance' if col == 'Performance_Communication_Absolute_Gap' else
                # 'Amplified risk score with classic greenwashing penalty' if col == 'Greenwashing_Risk_Amplified' else
                'Component score: Weakness of substantiation (0-100, higher = more risk)' if col == 'Component_Substantiation_Weakness_Score' else
                'Component score: Language Vagueness (0-100, higher = more risk)' if col == 'Component_Language_Vagueness_Score' else
                'Component score: Temporal orientation (0-100, higher = more risk)' if col == 'Component_Temporal_Orientation_Score' else
                'Component score: Reporting consistency (0-100, higher = more risk)' if col == 'Component_Reporting_Consistency_Score' else
                'Frequency of green terms as percentage of total words' if col == 'Green_Terms_Frequency_Pct' else
                'Unique green terms relative to document length' if col == 'Green_Terms_Unique_Relative' else
                'Combined score from green term frequency and uniqueness' if col == 'Green_Terms_Combined_Score' else
                'Overall sentiment score across all text' if col == 'Overall_Sentiment_Score' else
                'Average sentiment in renewable energy discussions' if col == 'Renewable_Energy_Sentiment' else
                'Average sentiment in climate/emissions discussions' if col == 'Climate_Emissions_Sentiment' else
                'Weighted combination of all sentiment scores' if col == 'Combined_Sentiment_Score' else
                'Intensity of quantitative claims and metrics' if col == 'Quantification_Intensity_Score' else
                'Intensity of evidence-based statements' if col == 'Evidence_Intensity_Score' else
                'Intensity of aspirational/future-oriented language' if col == 'Aspirational_Intensity_Score' else
                'Intensity of hedging language (uncertainty markers)' if col == 'Hedge_Intensity_Score' else
                'Intensity of vague, non-specific language' if col == 'Vague_Intensity_Score' else
                'Combined score of unclear language patterns' if col == 'Combined_Unclear_Intensity_Score' else
                'Percentage of commitments with specific timelines' if col == 'Commitment_Timeline_Pct' else
                'Ratio of future-focused vs past/present language' if col == 'Future_vs_Past_Present_Ratio' else
                'TF-IDF based document similarity score' if col == 'TFIDF_Document_Similarity' else
                'Jaccard similarity coefficient between documents' if col == 'Jaccard_Similarity' else
                'SpaCy semantic similarity average' if col == 'SpaCy_Average_Similarity' else
                'Ratio of highly similar sentences (SpaCy >0.8)' if col == 'SpaCy_High_Similarity_Ratio' else
                'Combined similarity score from multiple methods' if col == 'Similarity_Combined_Score' else
                f'Variable: {col}' for col in comprehensive_greenwashing.columns
            ],
            'Scale': [
                'Text' if col in ['Company'] else
                'Integer (2021, 2022)' if col == 'Year' else
                '0-100 (normalized by year)' if 'Score' in col or 'Component_' in col else
                'Percentage (0-100)' if 'Pct' in col else
                'Ratio (0+)' if 'Ratio' in col else
                'Normalized (0-100)' if any(x in col for x in ['Intensity', 'Similarity', 'Combined']) else
                'Score (-100 to +100)' if 'Sentiment' in col else
                'Continuous' for col in comprehensive_greenwashing.columns
            ]
        })
        
        data_dict.to_excel(writer, sheet_name='Data_Dictionary', index=False)
    
    print(f"✓ Comprehensive results exported successfully to: {output_path}")
    print(f"  - Comprehensive_Results: Complete dataset ({comprehensive_greenwashing.shape[0]} rows, {comprehensive_greenwashing.shape[1]} columns)")
    print(f"  - Results_2021: 2021 data only")
    print(f"  - Results_2022: 2022 data only") 
    print(f"  - Component_Methodology: Weights and calculation methods")
    print(f"  - Component_Summary: Component score means by year")
    print(f"  - Stats_2021/2022: Detailed descriptive statistics")
    print(f"  - Data_Dictionary: Complete variable descriptions")
    
except Exception as e:
    print(f"Error exporting comprehensive results: {e}")
    print("Comprehensive dataset created successfully in memory as 'comprehensive_greenwashing'")

# ==========================================
# STEP 5: Display key summary information
# ==========================================

print(f"\n" + "=" * 60)
print("COMPREHENSIVE GREENWASHING DATASET SUMMARY")
print("=" * 60)

print(f"\nDATASET OVERVIEW:")
print(f"  Total records: {len(comprehensive_greenwashing):,}")
print(f"  Companies: {comprehensive_greenwashing['Company'].nunique()}")
print(f"  Years: {sorted(comprehensive_greenwashing['Year'].unique())}")
print(f"  Variables: {comprehensive_greenwashing.shape[1]}")

print(f"\nVARIABLE CATEGORIES:")
main_scores = [col for col in comprehensive_greenwashing.columns if any(x in col for x in ['Performance_Communication_Gap_Score', 'Performance_Score', 'Green_Communication_Score', 'Absolute_Gap', 'Amplified'])]
components = [col for col in comprehensive_greenwashing.columns if 'Component_' in col]
green_terms = [col for col in comprehensive_greenwashing.columns if 'Green_Terms' in col or 'combined_green_score' in col]
sentiment = [col for col in comprehensive_greenwashing.columns if 'Sentiment' in col or 'sentiment' in col]
substantiation = [col for col in comprehensive_greenwashing.columns if any(x in col for x in ['Quantification', 'Evidence', 'Aspirational'])]
language = [col for col in comprehensive_greenwashing.columns if any(x in col for x in ['Hedge', 'Vague', 'Unclear', 'Timeline'])]
temporal = [col for col in comprehensive_greenwashing.columns if 'Future_vs_Past' in col]
similarity = [col for col in comprehensive_greenwashing.columns if any(x in col for x in ['Similarity', 'TFIDF', 'Jaccard', 'SpaCy'])]

print(f"  Main Scores: {len(main_scores)} variables")
print(f"  Component Scores: {len(components)} variables")
print(f"  Green Terms Analysis: {len(green_terms)} variables")
print(f"  Sentiment Analysis: {len(sentiment)} variables")
print(f"  Substantiation Metrics: {len(substantiation)} variables")
print(f"  Language Metrics: {len(language)} variables")
print(f"  Temporal Metrics: {len(temporal)} variables")
print(f"  Similarity/Consistency: {len(similarity)} variables")

print(f"\nKEY STATISTICS:")
print(f"  Mean Greenwashing Risk (2021): {comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2021]['Performance_Communication_Gap_Score'].mean():.2f}")
print(f"  Mean Greenwashing Risk (2022): {comprehensive_greenwashing[comprehensive_greenwashing['Year'] == 2022]['Performance_Communication_Gap_Score'].mean():.2f}")
print(f"  Companies with both years: {len(comprehensive_greenwashing.groupby('Company').filter(lambda x: len(x) == 2)['Company'].unique())}")

print(f"\nFILES CREATED:")
print(f"  Comprehensive dataset: {output_path}")
print(f"  (8 sheets with complete data, methodology, and documentation)")

print(f"\nREADY FOR ENSEMBLE ANALYSIS:")
print(f"  All component scores calculated and normalized")
print(f"  All underlying variables preserved")
print(f"  Complete documentation provided")

print(f"\nVariable 'comprehensive_greenwashing' is available in memory for immediate use")
print("Proceed to ensemble analysis with confidence that all data is preserved!")

# Final Score Calculation

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("ENSEMBLE GREENWASHING SCORE ANALYSIS")
print("="*50)

# ==========================================
# STEP 1: Generate valid weight combinations
# ==========================================

print("Generating valid weight combinations...")
print("Constraints:")
print("- Individual weights: 0.05 ≤ w ≤ 0.50")
print("- Greenwashing_Score > all other components")
print("- Substantiation_Weakness > Language_Vagueness, Temporal_Orientation, Reporting_Consistency")
print("- All weights sum to 1.0")
print()

# Create weight ranges (2 decimal precision as requested)
weight_range = np.arange(0.05, 0.51, 0.01)
weight_range = np.round(weight_range, 2)

valid_combinations = []
total_combinations = 0

# Generate all possible combinations
for w_gw in weight_range:  # Greenwashing_Score weight
    for w_sub in weight_range:  # Substantiation_Weakness weight
        for w_lang in weight_range:  # Language_Vagueness weight
            for w_temp in weight_range:  # Temporal_Orientation weight
                total_combinations += 1
                
                # Calculate remaining weight for Reporting_Consistency
                w_rep = 1.0 - (w_gw + w_sub + w_lang + w_temp)
                w_rep = round(w_rep, 2)
                
                # Check if all constraints are satisfied
                if (0.05 <= w_rep <= 0.50 and  # Reporting_Consistency in valid range
                    w_gw > w_sub and           # Greenwashing_Score > Substantiation_Weakness
                    w_gw > w_lang and          # Greenwashing_Score > Language_Vagueness 
                    w_gw > w_temp and          # Greenwashing_Score > Temporal_Orientation
                    w_gw > w_rep and           # Greenwashing_Score > Reporting_Consistency
                    w_sub > w_lang and         # Substantiation_Weakness > Language_Vagueness
                    w_sub > w_temp and         # Substantiation_Weakness > Temporal_Orientation  
                    w_sub > w_rep and          # Substantiation_Weakness > Reporting_Consistency
                    abs(w_gw + w_sub + w_lang + w_temp + w_rep - 1.0) < 0.001):  # Sum ≈ 1.0
                    
                    valid_combinations.append({
                        'w_greenwashing': w_gw,
                        'w_substantiation': w_sub,
                        'w_language': w_lang,
                        'w_temporal': w_temp,
                        'w_reporting': w_rep
                    })

print(f"Total combinations tested: {total_combinations:,}")
print(f"Valid combinations found: {len(valid_combinations):,}")
print(f"Percentage valid: {len(valid_combinations)/total_combinations*100:.2f}%")

if len(valid_combinations) == 0:
    print("ERROR: No valid weight combinations found. Check constraints.")
else:
    # ==========================================
    # STEP 2: Calculate scores for all combinations
    # ==========================================
    
    print(f"\nCalculating scores for {len(valid_combinations):,} weight combinations...")
    
    # Initialize storage for all results
    all_results = []
    
    # Progress bar for weight combinations
    for i, weights in enumerate(tqdm(valid_combinations, desc="Processing combinations")):
        
        # Calculate final score for this weight combination
        final_scores = (
            weights['w_greenwashing'] * greenwashing_df['Greenwashing_Score'] +
            weights['w_substantiation'] * greenwashing_df['Substantiation_Weakness'] +
            weights['w_language'] * greenwashing_df['Language_Vagueness'] +
            weights['w_temporal'] * greenwashing_df['Temporal_Orientation'] +
            weights['w_reporting'] * greenwashing_df['Reporting_Consistency']
        ).round(2)
        
        # Store results for this combination
        combination_result = {
            'combination_id': i,
            'weights': weights,
            'scores': final_scores.tolist(),
            'organizations': greenwashing_df['Organization'].tolist(),
            'years': greenwashing_df['year'].tolist(),
            'summary_stats': {
                'mean': final_scores.mean(),
                'median': final_scores.median(),
                'std': final_scores.std(),
                'min': final_scores.min(),
                'max': final_scores.max(),
                'q25': final_scores.quantile(0.25),
                'q75': final_scores.quantile(0.75)
            }
        }
        
        all_results.append(combination_result)
    
    # ==========================================
    # STEP 3: Create ensemble summary
    # ==========================================
    
    print("\nCreating ensemble summary...")
    
    # Extract all scores into matrix format
    n_combinations = len(all_results)
    n_observations = len(greenwashing_df)
    
    # Matrix: rows = combinations, columns = observations
    score_matrix = np.array([result['scores'] for result in all_results])
    
    # Calculate statistics across all combinations for each observation
    ensemble_stats = pd.DataFrame({
        'Organization': greenwashing_df['Organization'],
        'year': greenwashing_df['year'],
        'mean_score': np.mean(score_matrix, axis=0),
        'median_score': np.median(score_matrix, axis=0),
        'std_score': np.std(score_matrix, axis=0),
        'min_score': np.min(score_matrix, axis=0),
        'max_score': np.max(score_matrix, axis=0),
        'q25_score': np.percentile(score_matrix, 25, axis=0),
        'q75_score': np.percentile(score_matrix, 75, axis=0),
        'iqr_score': np.percentile(score_matrix, 75, axis=0) - np.percentile(score_matrix, 25, axis=0),
        'range_score': np.max(score_matrix, axis=0) - np.min(score_matrix, axis=0)
    }).round(2)
    
    # Weight distribution analysis
    weight_analysis = pd.DataFrame([result['weights'] for result in all_results])
    weight_stats = {
        'weight_ranges': {
            'greenwashing': [weight_analysis['w_greenwashing'].min(), weight_analysis['w_greenwashing'].max()],
            'substantiation': [weight_analysis['w_substantiation'].min(), weight_analysis['w_substantiation'].max()], 
            'language': [weight_analysis['w_language'].min(), weight_analysis['w_language'].max()],
            'temporal': [weight_analysis['w_temporal'].min(), weight_analysis['w_temporal'].max()],
            'reporting': [weight_analysis['w_reporting'].min(), weight_analysis['w_reporting'].max()]
        },
        'weight_means': weight_analysis.mean().to_dict(),
        'weight_stds': weight_analysis.std().to_dict()
    }
    
    # Overall ensemble statistics
    overall_stats = {
        'total_combinations': n_combinations,
        'total_observations': n_observations,
        'score_stability': {
            'mean_std_across_combinations': ensemble_stats['std_score'].mean(),
            'max_std_across_combinations': ensemble_stats['std_score'].max(),
            'mean_range_across_combinations': ensemble_stats['range_score'].mean(),
            'companies_with_high_uncertainty': len(ensemble_stats[ensemble_stats['std_score'] > ensemble_stats['std_score'].quantile(0.9)])
        }
    }
    
    # ==========================================
    # STEP 4: Calculate company averages across both years
    # ==========================================
    
    print("Calculating company averages across years...")
    
    # Company averages for median scores
    company_averages_median = ensemble_stats.groupby('Organization').agg({
        'median_score': ['mean', 'count']
    }).round(2)
    company_averages_median.columns = ['Avg_Median_Score', 'Years_Count']
    company_averages_median = company_averages_median.reset_index()
    company_averages_median = company_averages_median[company_averages_median['Years_Count'] == 2]  # Only companies with both years
    
    # Company averages for mean scores
    company_averages_mean = ensemble_stats.groupby('Organization').agg({
        'mean_score': ['mean', 'count']
    }).round(2)
    company_averages_mean.columns = ['Avg_Mean_Score', 'Years_Count']
    company_averages_mean = company_averages_mean.reset_index()
    company_averages_mean = company_averages_mean[company_averages_mean['Years_Count'] == 2]  # Only companies with both years
    
    # Combine company averages
    company_averages = pd.merge(company_averages_median[['Organization', 'Avg_Median_Score']], 
                               company_averages_mean[['Organization', 'Avg_Mean_Score']], 
                               on='Organization') 
    
    

In [ ]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill
from openpyxl import load_workbook

# ==========================================
# STEP 5: Save results
# ==========================================
if len(valid_combinations) == 0:
    print("ERROR: No valid weight combinations found. Check constraints.")
else:    
    print(f"\nSaving results...")
    
    # # Define file path
    # output_path = "data/Greenwashing Results/ensemble_results_summary_ensforperf.xlsx"
    
    # # Save key results as Excel for easy viewing
    # with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    #     ensemble_stats.to_excel(writer, sheet_name='Ensemble_Scores', index=False)
    #     company_averages.to_excel(writer, sheet_name='Company_Averages', index=False)
        
    #     # Summary statistics sheet
    #     summary_df = pd.DataFrame([
    #         ['Total Combinations', overall_stats['total_combinations']],
    #         ['Total Observations', overall_stats['total_observations']],
    #         ['Mean Std Across Combinations', overall_stats['score_stability']['mean_std_across_combinations']],
    #         ['Max Std Across Combinations', overall_stats['score_stability']['max_std_across_combinations']],
    #         ['Mean Range Across Combinations', overall_stats['score_stability']['mean_range_across_combinations']],
    #         ['High Uncertainty Companies', overall_stats['score_stability']['companies_with_high_uncertainty']]
    #     ], columns=['Metric', 'Value'])
    #     summary_df.to_excel(writer, sheet_name='Summary_Stats', index=False)
    
    # print("Applying Excel formatting...")
    
    # # Load the workbook for formatting
    # wb = load_workbook(output_path)
    
    # # Define grey fill for alternating rows
    # grey_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    
    # # Format each sheet
    # for sheet_name in wb.sheetnames:
    #     ws = wb[sheet_name]
        
    #     # Auto-adjust column widths based on the longest string in each column
    #     for col in ws.columns:
    #         max_length = 0
    #         col_letter = get_column_letter(col[0].column)
    #         for cell in col:
    #             if cell.value:
    #                 max_length = max(max_length, len(str(cell.value)))
    #         ws.column_dimensions[col_letter].width = max_length + 3  # Add padding
        
    #     # Apply alternating row colors
    #     if sheet_name in ['Ensemble_Scores', 'Company_Averages']:
    #         # These sheets have company names in column A - alternate by company
    #         prev_company = None
    #         use_grey = False
    #         for row in range(2, ws.max_row + 1):
    #             current_company = ws[f"A{row}"].value  # Column A has the company names
    #             if current_company != prev_company:
    #                 use_grey = not use_grey
    #                 prev_company = current_company
                
    #             if use_grey:
    #                 for col in range(1, ws.max_column + 1):
    #                     ws.cell(row=row, column=col).fill = grey_fill
    #     else:
    #         # Summary_Stats sheet - simple alternating rows
    #         for row in range(2, ws.max_row + 1):
    #             if row % 2 == 0:  # Even rows get grey background
    #                 for col in range(1, ws.max_column + 1):
    #                     ws.cell(row=row, column=col).fill = grey_fill
    
    # # Save the final formatted workbook
    # wb.save(output_path)
    
    # print(f"Results saved and formatted:")
    # print(f"- Summary tables: data/Greenwashing Results/ensemble_results_summary_ensforperf.xlsx")
    
    # ==========================================
    # STEP 6: Display results by year and averages
    # ==========================================
    
    print("\n" + "="*60)
    print("ENSEMBLE GREENWASHING SCORE RESULTS")
    print("="*60)
    
    print(f"\nOVERALL STATISTICS:")
    print(f"Total weight combinations tested: {overall_stats['total_combinations']:,}")
    print(f"Mean uncertainty (std) across all companies: {overall_stats['score_stability']['mean_std_across_combinations']:.2f}")
    print(f"Maximum uncertainty (std) for any company: {overall_stats['score_stability']['max_std_across_combinations']:.2f}")
    print(f"Companies with high score uncertainty (>90th percentile): {overall_stats['score_stability']['companies_with_high_uncertainty']}")
    
    # Results by year - 2021
    print(f"\n" + "="*40)
    print("HIGHEST RISK COMPANIES - 2021")
    print("="*40)
    print("(Ranked by median ensemble score)")
    
    ensemble_2021 = ensemble_stats[ensemble_stats['year'] == 2021]
    top_risk_2021 = ensemble_2021.nlargest(10, 'median_score')[['Organization', 'median_score', 'mean_score', 'std_score', 'min_score', 'max_score']]
    print(top_risk_2021.to_string(index=False))
    
    # Results by year - 2022
    print(f"\n" + "="*40)
    print("HIGHEST RISK COMPANIES - 2022")
    print("="*40)
    print("(Ranked by median ensemble score)")
    
    ensemble_2022 = ensemble_stats[ensemble_stats['year'] == 2022]
    top_risk_2022 = ensemble_2022.nlargest(10, 'median_score')[['Organization', 'median_score', 'mean_score', 'std_score', 'min_score', 'max_score']]
    print(top_risk_2022.to_string(index=False))
    
    # Company averages across both years
    print(f"\n" + "="*40)
    print("HIGHEST RISK COMPANIES - AVERAGE (2021-2022)")
    print("="*40)
    print("(Companies with data for both years)")
    
    top_avg_companies = company_averages.nlargest(14, 'Avg_Median_Score')
    print(top_avg_companies.to_string(index=False))
    
    print(f"\nCompanies with data for both years: {len(company_averages)}")
    
    # Most uncertain companies by year
    print(f"\n" + "="*40)
    print("MOST UNCERTAIN COMPANIES BY YEAR")
    print("="*40)
    print("(Highest std across weight combinations)")
    
    print(f"\n2021 - Most Uncertain:")
    most_uncertain_2021 = ensemble_2021.nlargest(5, 'std_score')[['Organization', 'median_score', 'mean_score', 'std_score', 'min_score', 'max_score']]
    print(most_uncertain_2021.to_string(index=False))
    
    print(f"\n2022 - Most Uncertain:")
    most_uncertain_2022 = ensemble_2022.nlargest(5, 'std_score')[['Organization', 'median_score', 'mean_score', 'std_score', 'min_score', 'max_score']]
    print(most_uncertain_2022.to_string(index=False))
    
    print(f"\n" + "="*60)
    print("ANALYSIS COMPLETE!")
    print("="*60)
    print("Use 'ensemble_stats' DataFrame for individual company/year scores")
    print("Use 'company_averages' DataFrame for cross-year company averages")
    print("All results saved to files for further analysis")

print("\nEnsemble analysis complete!")
print("\nKey variables created:")
print("- ensemble_stats: Individual company scores by year")
print("- company_averages: Company averages across both years") 
print("- weight_analysis: All valid weight combinations (in memory)")
print("- all_results: Complete results for all combinations")
print("\nFormatted results saved to: data/Greenwashing Results/ensemble_results_summary_ensforperf.xlsx")
print("(3 sheets: Ensemble_Scores, Company_Averages, Summary_Stats)")

## Sensitivity Analysis

In [ ]:
# Sensitivity Analysis Implementation
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import os

# Create directory for results
os.makedirs('data/Greenwashing Results/sensitivity_analysis', exist_ok=True)

print("COMMUNICATION COMPONENT SENSITIVITY ANALYSIS")
print("=" * 60)

# ==========================================
# STEP 1: Define Sensitivity Scenarios
# ==========================================

def create_sensitivity_scenarios():
    """Create all 17 sensitivity scenarios with hierarchy-preserving weight adjustments"""
    
    scenarios = {
        'baseline': {
            'name': 'Baseline',
            'description': 'Literature-based weights',
            'weights': {
                'combined_sentiment': [0.6, 0.2, 0.2],  # avg, renewable, climate
                'combined_green': [0.7, 0.3],  # freq, diversity
                'green_communication': [0.4, 0.6],  # green, sentiment
                'substantiation': [0.30, 0.35, 0.35],  # quantified, evidence, aspirational
                'language_Vagueness': [0.7, 0.3],  # vague, hedge
                'temporal': [0.6, 0.4],  # future, timeline
                'reporting': [0.7, 0.3],  # similarity, sentences
                'amplifier': 1.5
            }
        }
    }
    
    # Combined Sentiment variations (0.6-0.2-0.2)
    scenarios['sentiment_plus'] = {
        'name': 'Combined Sentiment +10%',
        'description': 'Primary sentiment weight increased',
        'weights': {**scenarios['baseline']['weights'],
                   'combined_sentiment': [0.66, 0.17, 0.17]}
    }
    scenarios['sentiment_minus'] = {
        'name': 'Combined Sentiment -10%',
        'description': 'Primary sentiment weight decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'combined_sentiment': [0.54, 0.23, 0.23]}
    }
    
    # Green Term variations (0.7-0.3)
    scenarios['green_term_plus'] = {
        'name': 'Green Term +10%',
        'description': 'Term frequency weight increased',
        'weights': {**scenarios['baseline']['weights'],
                   'combined_green': [0.77, 0.23]}
    }
    scenarios['green_term_minus'] = {
        'name': 'Green Term -10%',
        'description': 'Term frequency weight decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'combined_green': [0.63, 0.37]}
    }
    
    # Green Communication variations (0.4-0.6)
    scenarios['green_comm_plus'] = {
        'name': 'Green Communication +10%',
        'description': 'Sentiment priority increased',
        'weights': {**scenarios['baseline']['weights'],
                   'green_communication': [0.36, 0.64]}
    }
    scenarios['green_comm_minus'] = {
        'name': 'Green Communication -10%',
        'description': 'Sentiment priority decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'green_communication': [0.44, 0.56]}
    }
    
    # Substantiation variations (limited to ±5% to preserve hierarchy)
    scenarios['substantiation_plus'] = {
        'name': 'Substantiation +5%',
        'description': 'Evidence/aspirational weights increased',
        'weights': {**scenarios['baseline']['weights'],
                   'substantiation': [0.275, 0.3625, 0.3625]}
    }
    scenarios['substantiation_minus'] = {
        'name': 'Substantiation -5%',
        'description': 'Evidence/aspirational weights decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'substantiation': [0.325, 0.3375, 0.3375]}
    }
    
    # Language Vagueness variations (0.7-0.3)
    scenarios['language_plus'] = {
        'name': 'Language Vagueness +10%',
        'description': 'Vague language weight increased',
        'weights': {**scenarios['baseline']['weights'],
                   'language_Vagueness': [0.77, 0.23]}
    }
    scenarios['language_minus'] = {
        'name': 'Language Vagueness -10%',
        'description': 'Vague language weight decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'language_Vagueness': [0.63, 0.37]}
    }
    
    # Temporal variations (0.6-0.4)
    scenarios['temporal_plus'] = {
        'name': 'Temporal +10%',
        'description': 'Future orientation weight increased',
        'weights': {**scenarios['baseline']['weights'],
                   'temporal': [0.66, 0.34]}
    }
    scenarios['temporal_minus'] = {
        'name': 'Temporal -10%',
        'description': 'Future orientation weight decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'temporal': [0.54, 0.46]}
    }
    
    # Reporting variations (0.7-0.3)
    scenarios['reporting_plus'] = {
        'name': 'Reporting +10%',
        'description': 'Cross-year similarity weight increased',
        'weights': {**scenarios['baseline']['weights'],
                   'reporting': [0.77, 0.23]}
    }
    scenarios['reporting_minus'] = {
        'name': 'Reporting -10%',
        'description': 'Cross-year similarity weight decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'reporting': [0.63, 0.37]}
    }
    
    # Amplifier variations
    scenarios['amplifier_plus'] = {
        'name': 'Amplifier +10%',
        'description': 'Performance-communication gap amplifier increased',
        'weights': {**scenarios['baseline']['weights'],
                   'amplifier': 1.65}
    }
    scenarios['amplifier_minus'] = {
        'name': 'Amplifier -10%',
        'description': 'Performance-communication gap amplifier decreased',
        'weights': {**scenarios['baseline']['weights'],
                   'amplifier': 1.35}
    }
    
    return scenarios

scenarios = create_sensitivity_scenarios()
print(f"Created {len(scenarios)} sensitivity scenarios")

# ==========================================
# STEP 2: Recalculate Components for Each Scenario
# ==========================================

def recalculate_components_scenario(df_orig, weights):
    """Recalculate all communication components with modified weights"""
    
    df = df_orig.copy()
    
    # Recalculate combined sentiment score
    w_sent = weights['combined_sentiment']
    df['combined_sentiment_score'] = (
        w_sent[0] * df['avg_sentiment_score'] +     
        w_sent[1] * df['renewable_energy_avg_sentiment'] +
        w_sent[2] * df['climate_emissions_avg_sentiment']
    ).round(2)
    
    # Recalculate combined green score
    w_green = weights['combined_green']
    df['combined_green_score'] = (
        w_green[0] * df['gt_freq_pct'] +     
        w_green[1] * df['unique_gt_relative']
    ).round(2)
    
    # Recalculate Green Communication Score
    w_comm = weights['green_communication']
    df['Green_Com_Score'] = (
        w_comm[0] * df['combined_green_score'] +
        w_comm[1] * df['combined_sentiment_score']
    ).round(2)
    
    # Normalize Green_Com_Score per year to 0–100 scale
    normalize_by_year(df, 'Green_Com_Score')
    
    # Recalculate Substantiation Weakness
    w_sub = weights['substantiation']
    df['Substantiation_Weakness'] = (
        w_sub[0] * (100 - df['quantification_intensity_score']) +
        w_sub[1] * (100 - df['evidence_intensity_score']) +
        w_sub[2] * df['aspirational_intensity_score']
    ).round(2)
    normalize_by_year(df, 'Substantiation_Weakness')
    
    # Recalculate Language Vagueness
    w_lang = weights['language_Vagueness']
    df['Language_Vagueness'] = (
        w_lang[0] * df['vague_intensity_score'] +
        w_lang[1] * (100 - df['hedge_intensity_score'])
    ).round(2)
    normalize_by_year(df, 'Language_Vagueness')
    
    # Recalculate Temporal Orientation
    w_temp = weights['temporal']
    df['Temporal_Orientation'] = (
        w_temp[0] * df['future_vs_past_present_ratio'] +
        w_temp[1] * (100 - df['commitment_timeline_pct'])
    ).round(2)
    normalize_by_year(df, 'Temporal_Orientation')
    
    # Recalculate Reporting Consistency
    w_rep = weights['reporting']
    df['Reporting_Consistency'] = (
        w_rep[0] * df['similarity_combined'] +
        w_rep[1] * df['SpaCy_HighSim_Ratio']
    ).round(2)
    normalize_by_year(df, 'Reporting_Consistency')
    
    # Recalculate Greenwashing Score with modified amplifier
    amplifier = weights['amplifier']
    
    # Calculate absolute gap
    df['Greenwashing_Risk_Abs'] = (
        df['Green_Com_Score'] - df['Performance_Score']
    ).round(2)
    
    # Calculate yearly medians for classic greenwashing pattern
    yearly_medians = df.groupby('year').agg({
        'Performance_Score': 'median',
        'Green_Com_Score': 'median'
    })
    
    # Apply amplifier for classic greenwashing
    df['Amplified_Score'] = df.apply(
        lambda row: row['Greenwashing_Risk_Abs'] * amplifier if 
        ((row['Green_Com_Score'] > yearly_medians.loc[row['year'], 'Green_Com_Score']) and 
         (row['Performance_Score'] < yearly_medians.loc[row['year'], 'Performance_Score'])) 
        else row['Greenwashing_Risk_Abs'],
        axis=1
    ).round(2)
    
    # Normalize to 0–100 scale within each year
    normalize_by_year(df, 'Amplified_Score')
    df['Greenwashing_Score'] = df['Amplified_Score'].round(2)
    
    return df

# ==========================================
# STEP 3: Run Ensemble Analysis for Each Scenario
# ==========================================

def run_ensemble_for_scenario(df_scenario, scenario_name):
    """Run the ensemble methodology for a specific scenario using SAME method as original"""
    
    # Use the same ensemble weight combinations from the original analysis
    scenario_results = []
    
    # Use ALL valid combinations (same as original analysis)
    print(f"  Running ensemble with {len(valid_combinations):,} combinations...")
    
    # Calculate ensemble scores using ALL existing valid_combinations (same as original)
    for i, weights in enumerate(valid_combinations):
        
        # Calculate final score for this weight combination (EXACT same formula)
        final_scores = (
            weights['w_greenwashing'] * df_scenario['Greenwashing_Score'] +
            weights['w_substantiation'] * df_scenario['Substantiation_Weakness'] +
            weights['w_language'] * df_scenario['Language_Vagueness'] +
            weights['w_temporal'] * df_scenario['Temporal_Orientation'] +
            weights['w_reporting'] * df_scenario['Reporting_Consistency']
        ).round(2)
        
        scenario_results.append(final_scores.tolist())
    
    # Calculate ensemble statistics (EXACT same as original)
    score_matrix = np.array(scenario_results)
    
    scenario_ensemble = pd.DataFrame({
        'Organization': df_scenario['Organization'],
        'year': df_scenario['year'],
        'scenario': scenario_name,
        'mean_score': np.mean(score_matrix, axis=0),
        'median_score': np.median(score_matrix, axis=0),
        'std_score': np.std(score_matrix, axis=0),
        'min_score': np.min(score_matrix, axis=0),
        'max_score': np.max(score_matrix, axis=0),
        'q25_score': np.percentile(score_matrix, 25, axis=0),
        'q75_score': np.percentile(score_matrix, 75, axis=0),
        'iqr_score': np.percentile(score_matrix, 75, axis=0) - np.percentile(score_matrix, 25, axis=0),
        'range_score': np.max(score_matrix, axis=0) - np.min(score_matrix, axis=0)
    }).round(2)
    
    return scenario_ensemble

print(f"\nRunning ensemble analysis for each scenario...")
print(f"Note: Using all {len(valid_combinations):,} weight combinations per scenario")
print("This may take several minutes...")

# Store all scenario results
all_scenario_results = {}
scenario_counter = 0

for scenario_key, scenario_info in scenarios.items():
    scenario_counter += 1
    print(f"\nProcessing scenario {scenario_counter}/{len(scenarios)}: {scenario_info['name']}")
    
    # Recalculate components with modified weights
    df_scenario = recalculate_components_scenario(greenwashing_df, scenario_info['weights'])
    
    # Run ensemble analysis (this will take time due to all combinations)  
    scenario_ensemble = run_ensemble_for_scenario(df_scenario, scenario_info['name'])
    
    all_scenario_results[scenario_key] = scenario_ensemble
    print(f"  Completed: {scenario_info['name']}")

print("\nEnsemble analysis complete for all scenarios!")

# ==========================================
# STEP 4: Calculate Sensitivity Metrics
# ==========================================

# Combine all scenario results
combined_results = pd.concat(all_scenario_results.values(), ignore_index=True)

# Get baseline results for comparison
baseline_results = all_scenario_results['baseline'].copy()
baseline_rankings = baseline_results.groupby('year').apply(
    lambda x: x.sort_values('median_score', ascending=False).reset_index(drop=True)
).reset_index(drop=True)

print("\nCalculating sensitivity metrics...")

# Calculate ranking stability (R-bar-S) for each scenario
ranking_stability = {}
scenario_impact = []

for scenario_key, scenario_info in scenarios.items():
    if scenario_key == 'baseline':
        continue
        
    scenario_results = all_scenario_results[scenario_key]
    
    # Calculate R-bar-S (average ranking shift)
    total_rank_shifts = 0
    total_companies = 0
    
    for year in [2021, 2022]:
        baseline_year = baseline_results[baseline_results['year'] == year].copy()
        scenario_year = scenario_results[scenario_results['year'] == year].copy()
        
        # Rank companies by median score
        baseline_year['baseline_rank'] = baseline_year['median_score'].rank(method='dense', ascending=False)
        scenario_year['scenario_rank'] = scenario_year['median_score'].rank(method='dense', ascending=False)
        
        # Merge and calculate rank differences
        merged = baseline_year[['Organization', 'baseline_rank']].merge(
            scenario_year[['Organization', 'scenario_rank']], on='Organization'
        )
        
        rank_shifts = abs(merged['baseline_rank'] - merged['scenario_rank'])
        total_rank_shifts += rank_shifts.sum()
        total_companies += len(merged)
    
    r_bar_s = total_rank_shifts / total_companies if total_companies > 0 else 0
    ranking_stability[scenario_key] = r_bar_s
    
    # Calculate score impact metrics
    baseline_scores = baseline_results['median_score'].values
    scenario_scores = scenario_results['median_score'].values
    
    mad = np.mean(np.abs(scenario_scores - baseline_scores))
    cv = (np.std(scenario_scores - baseline_scores) / np.mean(baseline_scores)) * 100
    max_shift = np.max(np.abs(scenario_scores - baseline_scores))
    companies_large_shift = np.sum(np.abs(scenario_scores - baseline_scores) > 10)
    
    scenario_impact.append({
        'Scenario': scenario_info['name'],
        'R_bar_S': r_bar_s,
        'Avg_MAD': mad,
        'Avg_CV': cv,
        'Max_Score_Shift': max_shift,
        'Companies_Large_Shift': companies_large_shift,
        'Impact_Level': 'High' if r_bar_s > 4.0 else 'Moderate' if r_bar_s > 2.0 else 'Low'
    })

scenario_impact_df = pd.DataFrame(scenario_impact).sort_values('R_bar_S', ascending=False)

# ==========================================
# STEP 5: Company-Level Sensitivity Analysis  
# ==========================================

print("Analyzing company-level sensitivity...")

# Calculate sensitivity metrics for each company
company_sensitivity = []

for org in baseline_results['Organization'].unique():
    org_baseline = baseline_results[baseline_results['Organization'] == org]['median_score'].values
    
    org_scores = []
    for scenario_key in scenarios.keys():
        if scenario_key == 'baseline':
            continue
        scenario_scores = all_scenario_results[scenario_key]
        org_scenario = scenario_scores[scenario_scores['Organization'] == org]['median_score'].values  
        org_scores.extend(org_scenario)
    
    if len(org_scores) > 0 and len(org_baseline) > 0:
        # Calculate sensitivity metrics
        baseline_mean = np.mean(org_baseline)
        all_scores = np.array(org_scores + org_baseline.tolist())
        
        cv = (np.std(all_scores) / np.mean(all_scores)) * 100 if np.mean(all_scores) != 0 else 0
        score_range = np.max(all_scores) - np.min(all_scores)
        mad = np.mean(np.abs(org_scores - np.mean(org_baseline)))
        
        # Calculate ranking sensitivity
        rank_shifts = []
        for scenario_key in scenarios.keys():
            if scenario_key == 'baseline':
                continue
            scenario_results = all_scenario_results[scenario_key]
            for year in [2021, 2022]:
                if org in baseline_results[baseline_results['year'] == year]['Organization'].values:
                    baseline_rank = baseline_results[
                        (baseline_results['year'] == year)
                    ]['median_score'].rank(method='dense', ascending=False)[
                        baseline_results[
                            (baseline_results['year'] == year) & 
                            (baseline_results['Organization'] == org)
                        ].index[0]
                    ]
                    
                    scenario_rank = scenario_results[
                        (scenario_results['year'] == year)
                    ]['median_score'].rank(method='dense', ascending=False)[
                        scenario_results[
                            (scenario_results['year'] == year) & 
                            (scenario_results['Organization'] == org)
                        ].index[0]
                    ]
                    
                    rank_shifts.append(abs(baseline_rank - scenario_rank))
        
        avg_rank_shift = np.mean(rank_shifts) if rank_shifts else 0
        max_rank_shift = np.max(rank_shifts) if rank_shifts else 0
        
        company_sensitivity.append({
            'Organization': org,
            'Baseline_Score_Mean': baseline_mean,
            'CV_Percent': cv,
            'Score_Range': score_range,
            'MAD': mad,
            'Avg_Rank_Shift': avg_rank_shift,
            'Max_Rank_Shift': max_rank_shift,
            'Sensitivity_Level': 'High' if cv > 15 else 'Moderate' if cv > 5 else 'Low'
        })

company_sensitivity_df = pd.DataFrame(company_sensitivity).sort_values('CV_Percent', ascending=False)

print("Sensitivity analysis complete!")

# ==========================================
# STEP 6: Save Results
# ==========================================

print("\nSaving sensitivity analysis results...")

# Save comprehensive results
output_file = 'data/Greenwashing Results/sensitivity_analysis/communication_sensitivity_results.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Scenario impact summary
    scenario_impact_df.to_excel(writer, sheet_name='Scenario_Impact', index=False)
    
    # Company sensitivity analysis
    company_sensitivity_df.to_excel(writer, sheet_name='Company_Sensitivity', index=False)
    
    # All scenario results (sample)
    combined_results.to_excel(writer, sheet_name='All_Scenarios', index=False)
    
    # Baseline results for reference
    baseline_results.to_excel(writer, sheet_name='Baseline_Results', index=False)

print(f"Results saved to: {output_file}")

# ==========================================
# STEP 7: Display Key Results
# ==========================================

print("\n" + "="*60)
print("SENSITIVITY ANALYSIS RESULTS")
print("="*60)

print("\nSCENARIO IMPACT RANKING (by R-bar-S):")
print("="*40)
display_cols = ['Scenario', 'R_bar_S', 'Avg_MAD', 'Max_Score_Shift', 'Impact_Level']
print(scenario_impact_df[display_cols].head(10).to_string(index=False))

print(f"\nFRAMEWORK ROBUSTNESS ASSESSMENT:")
print("="*40)
avg_r_bar_s = scenario_impact_df['R_bar_S'].mean()
robust_scenarios = len(scenario_impact_df[scenario_impact_df['R_bar_S'] < 2.0])
print(f"Average R-bar-S across all scenarios: {avg_r_bar_s:.2f}")
print(f"Scenarios with high robustness (R-bar-S < 2.0): {robust_scenarios}/{len(scenario_impact_df)}")
print(f"Overall framework robustness: {'High' if avg_r_bar_s < 2.0 else 'Moderate' if avg_r_bar_s < 4.0 else 'Low'}")

print(f"\nMOST SENSITIVE COMPANIES:")
print("="*40)
sens_cols = ['Organization', 'CV_Percent', 'Score_Range', 'Max_Rank_Shift', 'Sensitivity_Level']
print(company_sensitivity_df[sens_cols].head(8).to_string(index=False))

print(f"\nCOMPANY SENSITIVITY DISTRIBUTION:")
print("="*40)
sensitivity_dist = company_sensitivity_df['Sensitivity_Level'].value_counts()
for level, count in sensitivity_dist.items():
    pct = (count / len(company_sensitivity_df)) * 100
    print(f"{level} sensitivity: {count} companies ({pct:.1f}%)")

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)
print("Key findings saved to Excel with multiple sheets")
print("Use scenario impact ranking to identify most influential methodological choices")
print("Use company sensitivity analysis to assess reliability of individual assessments")